<h1 style="text-align: center;">Tim "Poslednji ispit" - University of Kragujevac</h1>

<p style="text-align: center;">
  <img src="logopmfy.png" alt="University of Kragujevac Logo" width="200" height="200">
</p>

<h2 style="text-align: center;">Team Members</h2>

<ul style="list-style-type: none; text-align: center;">
  <li><strong>Vuk Lazović</strong></li>
  <li><strong>Sara Verlimirović</strong></li>
  <li><strong>Mihajlo Janković</strong></li>
</ul>


# Predstavljanje problema

.....

# Preuzimanje dataseta

In [ ]:
pip install -r requirements.txt

In [ ]:
import requests,os
from tqdm import tqdm


def check_dataset():
    files = os.listdir('.')
    flag=1
    for file in files:
        if("vehicles.csv"==file):
            flag=0
    return flag

def download():
    print("Downloading dataset.....")
    url = "https://www.dropbox.com/scl/fi/hiod02ra6fa1d5f5q7bmd/vehicles.csv?rlkey=ein4k3paqkw0ashh8njtyg6ed&st=gvyd3ohd&dl=1"
    response = requests.get(url, stream=True)
    
    # Get the total file size
    total_size = int(response.headers.get('content-length', 0))
    
    with open("vehicles.csv", mode="wb") as file:
        for chunk in tqdm(response.iter_content(chunk_size=10 * 1024), total=total_size//(10*1024), unit='KB'):
            file.write(chunk)

    response = requests.get(url, stream=True)


if(check_dataset()):
    download()
else:
    print("Dataset already downloaded")

In [ ]:
import pandas as pd
df=pd.read_csv("vehicles.csv")
df


In [ ]:
df.info()

In [ ]:
print(df.describe())

# Analiza

# Selekcija

# Modeli mašinsko učenja

## Resampling

## F-regression

## Decision Tree

## Random forest

## Neuronske mreže

# Zaključak

# Literatura

In [ ]:
pip freeze > requirements.txt